In [1]:
import cv2
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
from numpy import argmax
import os
from pickle import dump, load
import string
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

In [15]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

ROOT_DIR = "drive/MyDrive/Colab Notebooks"

CAPTION_GENERATOR_PATH = os.path.join(ROOT_DIR, "CaptionGenerator")
CAPTION_GENERATOR_DATASET_PATH = os.path.join(CAPTION_GENERATOR_PATH, "Dataset")
CAPTION_GENERATOR_TEXT_DATASET_PATH = os.path.join(CAPTION_GENERATOR_DATASET_PATH, "Text")
CAPTION_GENERATOR_TRAIN_MODEL_FIT_PATH = os.path.join(CAPTION_GENERATOR_DATASET_PATH, "fit")
CAPTION_GENERATOR_IMAGES_PATH = os.path.join(CAPTION_GENERATOR_DATASET_PATH, "img")
print(CAPTION_GENERATOR_TEXT_DATASET_PATH)

CAPTION_GENERATOR_DESCRIPTIONS_FILENAME = os.path.join(CAPTION_GENERATOR_DATASET_PATH, "descriptions.txt")
CAPTION_GENERATOR_FEATURES_FILENAME = os.path.join(CAPTION_GENERATOR_DATASET_PATH, "features.pkl")
CAPTION_GENERATOR_TOKENIZER_FILENAME = os.path.join(CAPTION_GENERATOR_DATASET_PATH, "tokenizer.pkl")

CAPTION_GENERATOR_TOKEN_FILENAME = os.path.join(CAPTION_GENERATOR_TEXT_DATASET_PATH, "Flickr8k.token.txt")
CAPTION_GENERATOR_TRAIN_IMAGES_FILENAME = os.path.join(CAPTION_GENERATOR_TEXT_DATASET_PATH, "Flickr_8k.trainImages.txt")
CAPTION_GENERATOR_TEST_IMAGES_FILENAME = os.path.join(CAPTION_GENERATOR_TEXT_DATASET_PATH, "Flickr_8k.devImages.txt")

CAPTION_GENERATOR_TRAINING_EPOCHS = 20

drive/MyDrive/Colab Notebooks/CaptionGenerator/Dataset/Text


In [18]:
class CaptionGenerator:
    
    IMAGE_WIDTH = 0
    IMAGE_HEIGHT = 0
    
    IMAGE_CLASSIFIER = None
    
    def __init__(self, image_width, image_height):
        self.IMAGE_WIDTH = image_width
        self.IMAGE_HEIGHT = image_height
        
        #self.IMAGE_CLASSIFIER = ImageClassifier(image_width, image_height)
        #self.IMAGE_CLASSIFIER.loadModel()
        
    def calculateDescriptionMaxLength(self, descriptions):
        lines = self.convertToLines(descriptions)
        return max(len(d.split()) for d in lines)
        
    def cleanDescriptions(self, descriptions):
        table = str.maketrans('', '', string.punctuation)
        for key, description_list in descriptions.items():
            for i in range(len(description_list)):
                description = description_list[i]
                description = description.split()
                description = [word.lower() for word in description]
                description = [w.translate(table) for w in description]
                description = [word for word in description if len(word)>1]
                description = [word for word in description if word.isalpha()]
                description_list[i] = ' '.join(description)
        
    def convertPath(self, path):
        validPath = ""
        for char in path:
            if char == '\\':
                validPath += "/"
            else:
                validPath += char
        return validPath
    
    def convertToLines(self, descriptions):
        all_descriptions = list()
        for key in descriptions.keys():
            [all_descriptions.append(d) for d in descriptions[key]]
        return all_descriptions
    
    def convertToVocabulary(self, descriptions):
        all_descriptions = set()
        for key in descriptions.keys():
            [all_descriptions.update(d.split()) for d in descriptions[key]]
        return all_descriptions
    
    def createSequences(self, tokenizer, max_length, description_list, photo, vocab_size):
        X1, X2, y = list(), list(), list()
        for description in description_list:
          sequence = tokenizer.texts_to_sequences([description])[0]
          for i in range(1, len(sequence)):
            in_seq, out_seq = sequence[:i], sequence[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
        return np.array(X1), np.array(X2), np.array(y)
    
    def createTokenizer(self, descriptions, verbose=True):
        lines = self.convertToLines(descriptions)
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(lines)
        vocab_size = len(tokenizer.word_index) + 1

        if verbose:
          print("Vocabulary size: %d" % vocab_size)

        return tokenizer, vocab_size
    
    def dataGenerator(self, descriptions, photos, tokenizer, max_length, vocab_size):
        while 1:
          for key, desc_list in descriptions.items():
            photo = photos[key][0]
            in_img, in_seq, out_word = self.createSequences(tokenizer, max_length, desc_list, photo, vocab_size)
            yield [in_img, in_seq], out_word
    
    def defineModel(self, vocab_size, max_length, verbose=False):
        # Feature extractor model
        inputs1 = Input(shape=(4096,))
        fe1 = Dropout(0.5)(inputs1)
        fe2 = Dense(256, activation="relu")(fe1)
        
        # Sequence model
        inputs2 = Input(shape=(max_length,))
        se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
        se2 = Dropout(0.5)(se1)
        se3 = LSTM(256)(se2)
        
        # Decoder model
        decoder1 = add([fe2, se3])
        decoder2 = Dense(256, activation="relu")(decoder1)
        outputs = Dense(vocab_size, activation="softmax")(decoder2)
        
        model = Model(inputs=[inputs1, inputs2], outputs=outputs)
        model.compile(loss="categorical_crossentropy", optimizer="adam")
        
        if verbose:
            print(model.summary())
            
        plot_model(model, to_file="model.png", show_shapes=True)
        
        return model
    
    def evaluateModel(self, model_filename):
        train_descriptions, train_features, tokenizer, vocab_size, max_length = self.prepareTrainingDataset()
        test_descriptions, test_features = self.prepareTestDataset()
        model = load_model("Dataset/fit/" + model_filename)
        
        actual, predicted = list(), list()
        for key, description_list in test_descriptions.items():
            yhat = self.generateDescription(model, tokenizer, test_features[key], max_length)
            references = [d.split() for d in description_list]
            actual.append(references)
            predicted.append(yhat.split())
            
        print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
        print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
        print("BLEU-3: %f" % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
        print("BLEU-4: %f" % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
        
    def extractFeatures(self, filename, output_file=CAPTION_GENERATOR_FEATURES_FILENAME, verbose=1):
        model = VGG16()
        model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
        if(verbose >= 2):
            print(model.summary())
            
        if os.path.isdir(filename):
            features = dict()
            for name in os.listdir(filename):
                filename = filename + '/' + name
                image = load_img(filename, target_size=(self.IMAGE_WIDTH, self.IMAGE_HEIGHT))
                image = img_to_array(image)
                image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
                image = preprocess_input(image)
                feature = model.predict(image, verbose=0)
                image_id = name.split('.')[0]
                features[image_id] = feature
                if(verbose >= 3):
                    print('>%s' % name) 
        
            if verbose >= 1:
                print("Extracted Features: %d" % len(features))
            
            dump(features, open(DATASET_PATH + output_file, 'wb'))
        
            return features
            
        elif os.path.isfile(filename):
            image = load_img(filename, target_size=(self.IMAGE_WIDTH, self.IMAGE_HEIGHT))
            image = img_to_array(image)
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = preprocess_input(image)
            feature = model.predict(image, verbose=0)
            return feature
    
    def fitCaptioningModel(self, train_descriptions, train_features, tokenizer, vocab_size, max_length, epochs=20):
        model = self.defineCaptioningModel(vocab_size, max_length, summarize=True)
        steps = len(train_descriptions)
        for i in range(epochs):
            generator = self.dataGenerator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
            model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
            model.save(os.path.join(CAPTION_GENERATOR_TRAIN_MODEL_FIT_PATH, str(i) + ".h5"))

    def fitTrainingModelWithProgressiveLoading(self, verbose=True):
        train_descriptions, train_features = self.loadTrainingDataset(verbose)
        tokenizer, vocab_size = self.createTokenizer(train_descriptions, verbose)
        max_length = self.getMaxLength(train_descriptions)
        model = self.defineModel(vocab_size, max_length, verbose)

        for epoch in range(CAPTION_GENERATOR_TRAINING_EPOCHS):
          generator = self.dataGenerator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
          model.fit(generator, epochs=1, steps_per_epoch=len(train_descriptions), verbose=1)
          model.save(os.path.join(CAPTION_GENERATOR_TRAIN_MODEL_FIT_PATH, "model_" + str(epoch) + ".h5"))
    
    def generateAndSaveTokenizer(self, 
                                 filename=CAPTION_GENERATOR_TOKENIZER_FILENAME,
                                 verbose=True, 
                                 descriptions_filename=CAPTION_GENERATOR_DESCRIPTIONS_FILENAME):
        train = self.loadSet(CAPTION_GENERATOR_TRAIN_IMAGES_FILENAME)
        if verbose:
            print("Dataset: %d" % len(train))
        
        train_descriptions = self.loadCleanDescriptions(train, filename=descriptions_filename)
        if verbose:
            print("Descriptions: train=%d" % len(train_descriptions))
        
        tokenizer = self.createTokenizer(train_descriptions)
        dump(tokenizer, open(filename, "wb"))
        
        if verbose:
            print("Tokenizer saved to file '" + filename + "'")
    
    def generateCaption(self, 
                        image_path, 
                        show_image=True, 
                        captioning_model=os.path.join(CAPTION_GENERATOR_TRAIN_MODEL_FIT_PATH, "model_0.h5"),
                        prediction_model_path="ImageClassifier/ImageClassifier.h5"):
    #    image_path = self.convertPath(image_path)
    #    image = self.readImageFile(image_path)
    #    self.IMAGE_CLASSIFIER.predict(image)
        print(os.path.join(CAPTION_GENERATOR_IMAGES_PATH, image_path))
        if not os.path.isfile(os.path.join(CAPTION_GENERATOR_IMAGES_PATH, image_path)):
            print("Invalid image path")
            return
    
        tokenizer = load(open(CAPTION_GENERATOR_TOKENIZER_FILENAME, "rb"))
        max_length = 34
        model = load_model(captioning_model)
        
        if show_image:
            image = plt.imread(image_path)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(image)
            plt.show()
            
        
        photo = self.extractFeatures(image_path)
        caption = self.generateDescription(model, tokenizer, photo, max_length)
        caption = caption[9: len(caption)-7]
            
        return caption
        
    def generateDescription(self, model, tokenizer, photo, max_length):
        in_text = "startseq"
        for i in range(max_length):
            sequence = tokenizer.texts_to_sequences([in_text])[0]
            sequence = pad_sequences([sequence], maxlen=max_length)
            yhat = model.predict([photo, sequence], verbose=0)
            yhat = argmax(yhat)
            word = self.wordFromId(yhat, tokenizer)
            if word is None:
                break
            
            in_text += ' ' + word
            
            if word == 'endseq':
                break
        return in_text

    def getMaxLength(self, descriptions):
        lines = self.convertToLines(descriptions)
        return max(len(description.split()) for description in lines)
        
    def getTextFileContent(self, filename):
        file = open(filename, 'r')
        text = file.read()
        file.close()
        return text
        
    def loadCleanDescriptions(self, 
                              dataset, 
                              descriptions_filename=CAPTION_GENERATOR_DESCRIPTIONS_FILENAME):
        doc = self.getTextFileContent(descriptions_filename)
        descriptions = dict()
        for line in doc.split('\n'):
            tokens = line.split()
            image_id, image_description = tokens[0], tokens[1:]
            if image_id in dataset:
                if image_id not in descriptions:
                    descriptions[image_id] = list()
                description = 'startseq ' + ' '.join(image_description) + ' endseq'
                descriptions[image_id].append(description)
        return descriptions

    def loadDescriptions(self, filename):
        text = self.getTextFileContent(filename)

        mapping = dict()
        for line in text.split('\n'):
            tokens = line.split()
            if len(line) < 2:
                continue
            image_id, image_description = tokens[0], tokens[1:]
            image_id = image_id.split('.')[0]
            image_description = ' '.join(image_description)
            if image_id not in mapping:
                mapping[image_id] = list()
            mapping[image_id].append(image_description)
        return mapping
    
    def loadPhotoFeatures(self, dataset, features_filename=CAPTION_GENERATOR_FEATURES_FILENAME):
        all_features = load(open(features_filename, 'rb'))
        features = {k: all_features[k] for k in dataset}
        return features
    
    def loadSet(self, filename):
        doc = self.getTextFileContent(filename)
        dataset = list()
        for line in doc.split('\n'):
            if len(line) < 1:
                continue
            identifier = line.split('.')[0]
            dataset.append(identifier)
        return set(dataset)

    def loadTestSet(self, verbose=True):
        test = self.loadSet(CAPTION_GENERATOR_TEST_IMAGES_FILENAME)
        test_descriptions = self.loadCleanDescriptions(test)
        test_features = self.loadPhotoFeatures(test)

        if verbose:
          print("Dataset: %d" % len(test))
          print("Descriptions: test=%d" % len(test_descriptions))
          print("Photos: test=%d" % len(test_features))

        return test_descriptions, test_features

    def loadTrainingDataset(self, verbose=True):
        train = self.loadSet(CAPTION_GENERATOR_TRAIN_IMAGES_FILENAME)
        train_descriptions = self.loadCleanDescriptions(train)
        train_features = self.loadPhotoFeatures(train)
        if verbose:
          print("Dataset: %d" % len(train))
          print("Descriptions: train=%d" % len(train_descriptions))
          print("Photos: train=%d" % len(train_features))

        return train_descriptions, train_features
    
    def prepareImageCaptioningTrainDataset(self):
        descriptions = self.loadDescriptions(CAPTION_GENERATOR_TOKEN_FILENAME)
        self.cleanDescriptions(descriptions)
        vocabulary = self.convertToVocabulary(descriptions)
        self.saveDescriptions(descriptions)
        
    def prepareTestDataset(self):
        filename = CAPTION_GENERATOR_TEST_IMAGES_FILENAME
        test = self.loadSet(filename)
        print("Dataset: %d" % len(test))
        
        test_descriptions = self.loadCleanDescriptions(test)
        print("Descriptions: test=%d" % len(test_descriptions))
        
        test_features = self.loadPhotoFeatures(CAPTION_GENERATOR_FEATURES_FILENAME, test)
        print("Photos: test=%d" % len(test_features))
        
        return test_descriptions, test_features
        
    def prepareTrainingDataset(self):
        filename = CAPTION_GENERATOR_TRAIN_IMAGES_FILENAME
        train = self.loadSet(filename)
        print("Dataset: %d" % len(train))
        
        train_descriptions = self.loadCleanDescriptions(train)
        print("Descriptions: train=%d" %len(train_descriptions))
        
        train_features = self.loadPhotoFeatures(train, CAPTION_GENERATOR_FEATURES_FILENAME)
        print("Photos: train=%d" % len(train_features))   
        
        tokenizer, vocab_size = self.createTokenizer(train_descriptions)
        print("Vocabulary size: %d" % vocab_size)
        
        max_length = self.calculateDescriptionMaxLength(train_descriptions)
        print("Description length: %d" % max_length)
        
        return train_descriptions, train_features, tokenizer, vocab_size, max_length
    
    def readImageFile(self, image_path):
        if not os.path.isfile(image_path):
            print("Unknow path")
            return None
        else:
            image = cv2.imread(image_path)
            image = cv2.resize(image, (self.IMAGE_WIDTH, self.IMAGE_HEIGHT))
            image = np.expand_dims(image, axis=0)
            return image
        
    def saveDescriptions(self, 
                         descriptions=None, 
                         filename=CAPTION_GENERATOR_DESCRIPTIONS_FILENAME,
                         verbose=True):
        print(filename)
        if descriptions == None:
          descriptions = self.loadDescriptions(CAPTION_GENERATOR_TOKEN_FILENAME)
          self.cleanDescriptions(descriptions)
          vocabulary = self.convertToVocabulary(descriptions)
          
          if verbose:
            print("Loaded: %d" % len(descriptions))
            print("Vocabulary size: %d" % len(vocabulary))

        lines = list()
        for key, description_list in descriptions.items():
            for description in description_list:
                lines.append(key + ' ' + description)
        data = '\n'.join(lines)
        file = open(filename, 'w')
        file.write(data)
        file.close()
        
    def wordFromId(self, integer, tokenizer):
        for word, index in tokenizer.word_index.items():
            if index == integer:
                return word
        return None

In [22]:
print(len(os.listdir("drive/MyDrive/Colab Notebooks/Dataset/Flicker8k_Dataset")))
print(os.listdir(DATASET_PATH))

FileNotFoundError: ignored

In [ ]:
captionGenerator = CaptionGenerator(IMAGE_WIDTH, IMAGE_HEIGHT)
#captionGenerator.saveDescriptions()

#train_descriptions, train_features = captionGenerator.loadTrainingDataset()

#tokenizer, vocab_size = captionGenerator.createTokenizer(train_descriptions)

#train_dataset, train_features, tokenizer, vocab_size, max_length = captionGenerator.prepareTrainingDataset()

captionGenerator.fitTrainingModelWithProgressiveLoading()

#captionGenerator.fitTrainingModelWithProgressiveLoading(verbose=True)

Dataset: 6000
Descriptions: train=6000
Photos: train=6000
Vocabulary size: 7579
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 256)      1940224     input_4[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 4096)         0           input_3[0][0]                    
____________

In [39]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)
 
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features
 
# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return array(X1), array(X2), array(y)
 
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	# compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	model.summary()
	plot_model(model, to_file='model.png', show_shapes=True)
	return model
 
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield [in_img, in_seq], out_word
 
# load training dataset (6K)
filename = TRAIN_IMAGES_FILENAME
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions(DESCRIPTIONS_FILENAME, train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features(FEATURES_FILENAME, train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)
 
# define the model
model = define_model(vocab_size, max_length)
# train the model, run epochs manually and save after each epoch
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
	# create the data generator
	generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
	# fit for one epoch
	model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
	# save model
	model.save('model_' + str(i) + '.h5')

Dataset: 6000
Descriptions: train=6000
Photos: train=6000
Vocabulary Size: 7579
Description Length: 34
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 4096)]       0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 34, 256)      1940224     input_16[0][0]                   
__________________________________________________________________________________________________
dropout_14 (Dropout)            (None, 4096)         0           input_15[0][0]         

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


  46/6000 [..............................] - ETA: 29:19 - loss: 7.5338

KeyboardInterrupt: ignored

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
